In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import time
import os
import csv

In [2]:
#<Path>
path = '/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/5_Similarity/'
file_name = ['sorted_score_vector_except_1_aa', 'sorted_score_vector_except_1_ab', 'sorted_score_vector_except_1_ac', 'sorted_score_vector_except_1_ad', 'sorted_score_vector_except_1_ae', 'sorted_score_vector_except_1_af', 'sorted_score_vector_except_1_ag', 'sorted_score_vector_except_1_ah', 'sorted_score_vector_except_1_ai', 'sorted_score_vector_except_1_aj']
score_path = list()
for i in file_name:
    score_path.append(path + i)


In [5]:
def file_open(file_path):
    print time.strftime('%a %H:%M:%S')
    data = []
    with open(file_path) as f:
        for line in f:
            data.append(line.split(','))
    print time.strftime('%a %H:%M:%S')
    return data

In [6]:
score = list()
for i in score_path:
    score = score + file_open(i)
    print 'check'

Wed 08:49:15
Wed 08:50:38
check
Wed 08:50:42
Wed 08:54:35
check
Wed 08:54:40
Wed 08:58:37
check
Wed 08:58:45
Wed 09:02:27
check
Wed 09:02:37
Wed 09:06:40
check
Wed 09:06:52
Wed 09:11:02
check
Wed 09:11:14
Wed 09:13:16
check
Wed 09:13:29
Wed 09:16:37
check
Wed 09:16:55
Wed 09:21:51
check
Wed 09:22:10
Wed 09:25:41
check


In [7]:
len(score) # 원래는 65783688인데..(이것도 아님) ㅠ)_ㅠ

646688125

In [67]:
score[0]

['80303', 'HMDB37761', '0.9967105263157895\r\n']

In [14]:
#<FIle open>
#f_score = open(score_path, 'r')
# it takes about  mins at shepherd 6
# data : 
#score = [] #646,703,866
#print time.strftime('%a %H:%M:%S')
#while True:
#    line = f_score.readline()
#    if not line: break
#    score.append(line.split(','))
#print time.strftime('%a %H:%M:%S')

Wed 17:01:07
Wed 17:03:51


In [16]:
#len(score)

65783688

In [8]:
# aceept user input
cutoff_percentage = input('Enter the percentage(ex. 0.1) : ')

Enter the percentage(ex. 0.1) : 0.1


In [9]:
# extract element over cutoff. cutoff is top 0.0001 per cent of wo1.
cutoff_idx = int(len(score)*cutoff_percentage / 100)
cutoff = score[cutoff_idx] # 0.9939

similar_comp = []
for item2 in score[:cutoff_idx]: 
    similar_comp.append(item2)

In [ ]:
############ top hmdb의 smiles와 kegg substrate의 smiles를 비교해서 ID를 convert한다.

In [10]:
# read hmdb file ['Accession','Name','ChemSpider_InChI key','ChemSpider_InChI','ChemSpider_Smiles']
f_hmdb = open('/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/4_HMDB/HMDB_ChemSpider(name)_txt.txt', 'r')
hmdb = []
print time.strftime('%a %H:%M:%S')
while True:
    line = f_hmdb.readline()
    if not line: break
    hmdb.append(line.split('\t'))
print time.strftime('%a %H:%M:%S')

Wed 09:36:01
Wed 09:36:15


In [11]:
f_c_c1 = open('/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/3_ChemSpider/KEGG_SID_CID_SMILES(PubChem)_SMILES(ChemSpider)_C1.csv', 'r')
# read KEGG Compound1 file [KEGG ID, SID, CID, PubChem Smiles, Chemspider Smiles]
kegg_c1 = []
print time.strftime('%a %H:%M:%S')
while True:
    line = f_c_c1.readline()
    if not line: break
    kegg_c1.append(line.split(','))
print time.strftime('%a %H:%M:%S')

Wed 09:36:19
Wed 09:36:19


In [12]:
# extract hmdb_id of data ( top 0.0001% similar compound)
top_hmdb_id = []
for i in similar_comp: # decending order
    top_hmdb_id.append(i[1].rstrip('\r\n'))
top_hmdb_id_rm_dup = set(top_hmdb_id)

In [13]:
# extract all hmdb_id in HMDB database
all_hmdb_id = []
for i in hmdb[1:]: # first line is header
    all_hmdb_id.append(i[0].rstrip('\r\n'))

In [14]:
# find the index of hmdb_id in all hmdb_id
print time.strftime('%a %H:%M:%S')
top_hmdb_idx_rm_dup = []
for i in top_hmdb_id_rm_dup:
    top_hmdb_idx_rm_dup.append(all_hmdb_id.index(i))
print time.strftime('%a %H:%M:%S')

Wed 09:36:25
Wed 09:36:34


In [15]:
# extract Smiles of hmdb_id(top 0.0001% compound) from all hmdb database
top_hmdb_smiles_rm_dup = []
for i in top_hmdb_idx_rm_dup:
    top_hmdb_smiles_rm_dup.append(hmdb[i+1][4].rstrip('\r\n')) # The first line of hmdb is header. That's why do i+1

In [16]:
# Extract KEGG ID
k_c1_id = []
for i in kegg_c1[1:]:
    k_c1_id.append(i[1])

In [17]:
# Extract ChemSpider Smiles of KEGG compound1
k_c1_smiles = []
for i in kegg_c1[1:]:
    k_c1_smiles.append(i[6].rstrip('\n'))

In [18]:
def indices(lst, element):
    result = []
    offset = -1
    while True:
        try:
            offset = lst.index(element, offset+1)
        except ValueError:
            return result
        result.append(offset)

In [19]:
# pairing the hmdb_id and KEGG ID using smiles (It means converting the hmdb id to kegg id)
h_k_pair = []
h_idx = -1
print time.strftime('%a %H:%M:%S')
for i in top_hmdb_smiles_rm_dup:
    h_idx = h_idx + 1
    try :
        k_idx = indices(k_c1_smiles, i)
        for j in k_idx:
            h_k_pair.append([top_hmdb_id[h_idx], k_c1_id[j]]) # [HMDB ID, KEGG ID]
    except ValueError:
        continue
print time.strftime('%a %H:%M:%S')

Wed 09:36:42
Wed 09:36:44


In [20]:
h_k_pair[0]

['HMDB37428', 'C00431']

In [21]:
f_k_c1 = open('/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/1_KEGG_rpair/KEGG_C1.csv', 'r')
f_k_c2 = open('/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/1_KEGG_rpair/KEGG_C2.csv', 'r')

In [22]:
def read_csv_file(f):
    f.seek(0)
    temp = []
    print time.strftime('%a %H:%M:%S')
    while True:
        line = f.readline()
        if not line: break
        temp.append(line.split(','))
    print time.strftime('%a %H:%M:%S')
    return temp

In [ ]:
# read compound1 from kegg rpair
sub = read_csv_fie(f_k_c1)

# read compound2 from kegg rpair
pro = read_csv_fie(f_k_c2)

substrate = []
for i in sub[1:]:
    substrate.append(i[0].rstrip('\n'))

product = []
for i in pro[1:]:
    product.append(i[0].rstrip('\n'))    

In [24]:
# find substrate(which is converted from hmdb compound)-product pair
hmdb_sub_pro = []
for i in h_k_pair:
    idxs = indices(substrate, i[1])
    for j in idxs:
        hmdb_sub_pro.append(i + [product[j]])

In [57]:
print len(h_k_pair)
print len(hmdb_sub_pro)

948
4657


In [35]:
len(hmdb_sub_pro)

4657

In [25]:
#################Herbal compound

In [26]:
top_herb_id = []
for i in similar_comp:
    top_herb_id.append(int(i[0]))
top_herb_id_rm_dup = set(top_herb_id)

In [40]:
top_herb_id_rm_dup

{32768,
 32769,
 65539,
 32772,
 32773,
 65542,
 65543,
 65544,
 32777,
 70999,
 32780,
 65550,
 65551,
 32785,
 32786,
 32771,
 65557,
 65558,
 65559,
 32792,
 65561,
 65565,
 65541,
 65568,
 32802,
 32806,
 32807,
 32808,
 32814,
 32816,
 32819,
 32824,
 32825,
 32826,
 71007,
 65598,
 65599,
 32833,
 65602,
 65603,
 32836,
 32837,
 65606,
 32839,
 65608,
 65548,
 65611,
 65613,
 32846,
 32847,
 32851,
 65624,
 32857,
 65627,
 65628,
 32865,
 65635,
 65636,
 32873,
 32874,
 32875,
 65646,
 65647,
 32880,
 65650,
 32883,
 65652,
 65653,
 65655,
 32888,
 65658,
 65659,
 65661,
 32789,
 65664,
 32899,
 32900,
 32790,
 32904,
 32905,
 32907,
 32908,
 32909,
 65560,
 32916,
 65689,
 32923,
 65694,
 32931,
 32932,
 32933,
 32935,
 65704,
 32938,
 32939,
 65708,
 32941,
 65712,
 65715,
 65716,
 34922,
 32951,
 32952,
 65721,
 32954,
 32955,
 32956,
 32957,
 65727,
 32960,
 32961,
 65731,
 32966,
 65735,
 65736,
 65737,
 65739,
 65740,
 65741,
 32975,
 65744,
 65745,
 32978,
 32979,
 65748,


In [36]:
f_herb_ids = open('/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/7_Ezyme_tool/1-5_herb_kegg(final).tsv', 'r')

In [41]:
def read_delimited_file(f,delimiter):
    f.seek(0)
    temp = []
    print time.strftime('%a %H:%M:%S')
    while True:
        line = f.readline()
        if not line: break
        temp.append(line.split(delimiter))
    f.close()
    print time.strftime('%a %H:%M:%S')
    return temp

In [42]:
# read files
ids = read_delimited_file(f_herb_ids, '\t')

Wed 09:51:07
Wed 09:51:07


In [118]:
# extract herb compID, KEGGID
herb_comp_id = list() #3391
herb_kegg_id = list() #3391
for i in ids[1:]: # start with 1 because first line is the column name
    herb_comp_id.append(int(i[3].rstrip()))
    herb_kegg_id.append(i[9].rstrip())
    

In [79]:
hmdb_final = list()
for i in h_k_pair:
    hmdb_final.append(i[0])

In [80]:
len(hmdb_final)

948

In [81]:
print len(top_hmdb_id)
print len(top_herb_id)

646688
646688


In [63]:
hmdb_similar_herb = list()
#hmdb_final의 simialr한 herb comp가 kegg id를 가지고 있는지 확인해야함.
for i in hmdb_final:
    idx = indices(top_hmdb_id, i)
    subset = list()
    for j in idx:
        subset.append(top_herb_id[j])
    hmdb_similar_herb.append((i, subset))

In [128]:
hmdb_herb_kegg = list()

for i in hmdb_similar_herb:
    hmdb_id = i[0]
    herb_ids = i[1]
    for herb_id in herb_ids:
        idx = indices(herb_comp_id, herb_id)
        if idx != []:
            for j in idx:
                hmdb_herb_kegg.append((hmdb_id, herb_id, herb_kegg_id[j]))
    

In [131]:
# 여기 너무 느림. 속도 개선 필요
hmdb_herb_kegg = list()
subset = list()
for i in hmdb_similar_herb:
    hmdb_id = i[0]
    herb_ids = i[1]
    for herb_id in herb_ids:
        idx = indices(herb_comp_id, herb_id)
        if idx != []:
            subset.append((hmdb_id, herb_id, idx))

for j in subset:
    hmdb_id = j[0]
    herb_id = j[1]
    idxs = j[2]
    for k in idxs:
        hmdb_herb_kegg.append((hmdb_id, herb_id, herb_kegg_id[k]))

In [132]:
hmdb_herb_kegg # substrate가 존재하는 hmdb와, 그 hmdb와 비슷한 herb id와 그 herb id의 kegg id들.

[('HMDB37428', 67664, 'C12634'),
 ('HMDB37428', 60628, 'C16981'),
 ('HMDB37428', 88455, 'C16981'),
 ('HMDB37428', 88603, 'C12634'),
 ('HMDB37428', 63325, 'C17600'),
 ('HMDB37428', 75593, 'C17600'),
 ('HMDB37428', 88970, 'C12637'),
 ('HMDB37428', 66187, 'C19796'),
 ('HMDB37428', 88968, 'C12667'),
 ('HMDB37428', 63328, 'C17563'),
 ('HMDB37428', 63636, 'C05625'),
 ('HMDB37428', 51660, 'C12249'),
 ('HMDB37428', 69046, 'C12626'),
 ('HMDB37428', 79418, 'C12626'),
 ('HMDB37428', 71552, 'C12249'),
 ('HMDB37428', 70847, 'C16911'),
 ('HMDB37428', 88452, 'C16911'),
 ('HMDB37428', 66211, 'C01750'),
 ('HMDB37428', 85489, 'C10108'),
 ('HMDB37428', 70357, 'C10108'),
 ('HMDB37428', 66812, 'C05625'),
 ('HMDB37428', 84943, 'C08725'),
 ('HMDB37428', 92673, 'C12644'),
 ('HMDB37428', 54315, 'C16312'),
 ('HMDB37428', 79364, 'C16306'),
 ('HMDB37428', 53822, 'C08639'),
 ('HMDB37428', 68831, 'C17140'),
 ('HMDB37428', 65022, 'C08727'),
 ('HMDB37428', 67016, 'C08064'),
 ('HMDB37428', 10197, 'C17555'),
 ('HMDB374

In [149]:
# HMDB id , substrate id (kegg), herb id, herb kegg id, product id(kegg)
hmdb_sub_pro_id = list()
for i in hmdb_sub_pro:
    hmdb_sub_pro_id.append(i[0])
    
    

In [152]:
hmdb_sub_herb_herbKegg_pro = list()
for i in hmdb_herb_kegg:
    idxs = indices(hmdb_sub_pro_id, i[0])
    for j in idxs:
        hmdb_sub_herb_herbKegg_pro.append((i[0], hmdb_sub_pro[j][1],i[1], i[2], hmdb_sub_pro[j][2]))

In [153]:
hmdb_sub_herb_herbKegg_pro

[('HMDB37428', 'C00431', 67664, 'C12634', 'C00990'),
 ('HMDB37428', 'C00431', 67664, 'C12634', 'C03273'),
 ('HMDB37428', 'C00431', 67664, 'C12634', 'C03087'),
 ('HMDB37428', 'C00431', 67664, 'C12634', 'C00763'),
 ('HMDB37428', 'C00757', 67664, 'C12634', 'C03329'),
 ('HMDB37428', 'C00757', 67664, 'C12634', 'C01795'),
 ('HMDB37428', 'C00757', 67664, 'C12634', 'C11818'),
 ('HMDB37428', 'C00431', 60628, 'C16981', 'C00990'),
 ('HMDB37428', 'C00431', 60628, 'C16981', 'C03273'),
 ('HMDB37428', 'C00431', 60628, 'C16981', 'C03087'),
 ('HMDB37428', 'C00431', 60628, 'C16981', 'C00763'),
 ('HMDB37428', 'C00757', 60628, 'C16981', 'C03329'),
 ('HMDB37428', 'C00757', 60628, 'C16981', 'C01795'),
 ('HMDB37428', 'C00757', 60628, 'C16981', 'C11818'),
 ('HMDB37428', 'C00431', 88455, 'C16981', 'C00990'),
 ('HMDB37428', 'C00431', 88455, 'C16981', 'C03273'),
 ('HMDB37428', 'C00431', 88455, 'C16981', 'C03087'),
 ('HMDB37428', 'C00431', 88455, 'C16981', 'C00763'),
 ('HMDB37428', 'C00757', 88455, 'C16981', 'C03

In [155]:
len(hmdb_sub_herb_herbKegg_pro)

187610

In [164]:
import requests

In [161]:
url

'http://rest.genome.jp/ezyme/C12634/C00990'

In [199]:
# using UniChem REST API, convert pubchem ID to KEGG ID
# 1. requests HTML

url_base = 'http://rest.genome.jp/ezyme/'

# list of r object(request result)
#rs=[]
#cnt = 0
print time.strftime('%a %H:%M:%S')
for i in hmdb_sub_herb_herbKegg_pro[len(rs):]:
    try:
        url = url_base + i[3] +'/' + i[4]
        #scrap the HTML at the url
        r = requests.get(url)
        #rows = r.text.split('\n')
        #row = rows[1].split('\t')
        #predicted_score = row[0].encode('ascii', 'replace')
        #predicted_orthology = row[1].encode('ascii', 'replace')
        #rs.append(i+(predicted_orthology, predicted_score))
        rs.append(r)
        r.close()
    except requests.ConnectionError:
        #rs.append('null')
    #except urllib2.socket.error:
        print i
        print len(rs)
        print url
        print r
        print rs[len(rs)-1]
        break   
    cnt = cnt + 1
    if cnt % 1000 == 0 :
        print '%d : '% cnt + time.strftime('%a %H:%M:%S')
print time.strftime('%a %H:%M:%S')

Thu 09:38:21
('HMDB40294', 'C14871', 66187, 'C19796', 'C14873')
976
http://rest.genome.jp/ezyme/C19796/C14873
<Response [200]>
<Response [200]>
Thu 09:38:25


In [186]:
print cnt
print len(rs)

975
975


In [171]:
type(r.text)
r.text.split('\n')[0]

u'# Socre\tKO\tOCs\tReference RPAIRs'

In [172]:
a= r.text.split('\n')[1]
a

u'0.050\tK11782\tOC.1052773_b, OC.1052777_b, OC.500479_ba, OC.1052774_b, OC.939560_a, OC.1001548_b, OC.937445_a\tRP14031'

In [173]:
b = a.split('\t')

[u'0.050',
 u'K11782',
 u'OC.1052773_b, OC.1052777_b, OC.500479_ba, OC.1052774_b, OC.939560_a, OC.1001548_b, OC.937445_a',
 u'RP14031']

In [191]:
f_temp = open('/home/share/aschoi/nas/users/asolchoi/data/COCONUT_Herbal/4_HMDB/HMDB_PubChem_KEGG.tsv')

In [192]:
temp = read_delimited_file(f_temp, '\t')

Thu 08:52:29
Thu 08:52:29


In [193]:
temp

[['Accession', 'Name', 'PubChem_CID', 'KEGG_ID\n'],
 ['HMDB00002', '1,3-Diaminopropane', '428.0', 'C00986\n'],
 ['HMDB00005', '2-Ketobutyric acid', '58.0', 'C00109\n'],
 ['HMDB00010', '2-Methoxyestrone', '440624.0', 'C05299\n'],
 ['HMDB00011', '(R)-3-Hydroxybutyric acid', '92135.0', 'C01089\n'],
 ['HMDB00012', 'Deoxyuridine', '13712.0', 'C00526\n'],
 ['HMDB00014', 'Deoxycytidine', '13711.0', 'C00881\n'],
 ['HMDB00016', 'Deoxycorticosterone', '6166.0', 'C03205\n'],
 ['HMDB00017', '4-Pyridoxic acid', '6723.0', 'C00847\n'],
 ['HMDB00019', 'Alpha-ketoisovaleric acid', '49.0', 'C00141\n'],
 ['HMDB00020', 'p-Hydroxyphenylacetic acid', '127.0', 'C00642\n'],
 ['HMDB00021', 'Iodotyrosine', '439744.0', 'C02515\n'],
 ['HMDB00022', '3-Methoxytyramine', '1669.0', 'C05587\n'],
 ['HMDB00023', '(S)-3-Hydroxyisobutyric acid', '440873.0', 'C06001\n'],
 ['HMDB00026', 'Ureidopropionic acid', '111.0', 'C02642\n'],
 ['HMDB00030', 'Biotin', '171548.0', 'C00120\n'],
 ['HMDB00032', '7-Dehydrocholesterol', '439

In [195]:
temp_acc = list()
for i in temp[1:]:
    temp_acc.append(i[0].rstrip())

In [197]:
count = list()
for i in top_hmdb_id_rm_dup:
    try :
        count.append(temp_acc.index(i))
    except:
        continue

In [198]:
len(count)

1130

In [146]:
len(top_hmdb_id_rm_dup)

32410

In [147]:
len(h_k_pair)

948

In [148]:
len(top_hmdb_id)

646688